In [8]:
# audio_path = r"C:\Users\Tomek\Desktop\02. A Broken Frame (1982)\02. Odyssey (2014).mp3"
# images_path = r"E:\Muzyka SD\assign"
del_path = r"E:\Muzyka SD\Jimi Hendrix"

In [9]:
from mutagen.easyid3 import EasyID3
from mutagen.flac import FLAC, Picture
from mutagen.id3 import ID3, APIC, error
from os import path, chdir, listdir, getcwd

desktop_path = path.join(path.expanduser("~"), "Desktop")
images_list = []
images_list_no_ext = []

def RemoveImageMP3(mp3_path):
    """
    Removes an image from a mp3 file.

    Args:
        mp3_path (str): Path of a mp3 file.
    """
    try:
        audio = ID3(mp3_path)

        # Remove all APIC (attached picture) frames
        audio.delall("APIC")
        audio.save()
        # print(f"All embedded images removed from {file_path}")
    except error as e:
        print(f"Failed to remove images: {e}")

def AddImageMP3(mp3_path, image_path):
    """
    Adds an image to a mp3 file.

    Args:
        mp3_path (str):     Path of a mp3 file.
        image_path (str):   Path of an image.
    """
    try:
        audio = ID3(mp3_path)
        with open(image_path, 'rb') as img:
            audio['APIC'] = APIC(encoding=3,         # 3 is for utf-8
                                 mime='image/jpeg',  # image type, you can use image/png or others
                                 type=3,             # 3 is for the cover (front) image
                                 desc=u'Cover',
                                 data=img.read()
                                )
        audio.save()
        # print(f"New image added to {file_path}")
    except error as e:
        print(f"Failed to add image: {e}")

def RemoveImageFLAC(flac_path):
    """
    Removes an image from a flac file.

    Args:
        flac_path (str): Path of a flac file.
    """
    try:
        audio = FLAC(flac_path)
        
        # Remove all pictures from the FLAC file
        audio.clear_pictures()
        audio.save()
        # print(f"All embedded images removed from {flac_path}")
    except Exception as e:
        print(f"Failed to remove images: {e}")

def AddImageFLAC(flac_path, image_path):
    """
    Adds an image to a flac file.

    Args:
        flac_path (str):    Path of a flac file.
        image_path (str):   Path of an image.
    """
    try:
        audio = FLAC(flac_path)
        image = Picture()
        with open(image_path, 'rb') as img:
            image.data = img.read()
        
        image.type = 3  # Cover (front)
        image.mime = "image/jpeg" if image_path.lower().endswith(".jpg") else "image/png"
        image.desc = "Cover"
        image.width = 0  # Optional: set image dimensions, if known
        image.height = 0
        image.depth = 0

        # Add the picture to the FLAC file
        audio.add_picture(image)
        audio.save()
        # print(f"New image added to {flac_path}")
    except Exception as e:
        print(f"Failed to add image: {e}")

def RemoveAndAddImageToAudiofile(audio_path, image_path):
    """
    Removes, than adds an image to a mp3 or flac file.

    Args:
        audio_path (str): Path of a audio file.
        image_path (str): Path of an image.
    """
    if audio_path.endswith(".mp3"):
        RemoveImageMP3(audio_path)
        AddImageMP3(audio_path, image_path)
    elif audio_path.endswith(".flac"):
        RemoveImageFLAC(audio_path)
        AddImageFLAC(audio_path, image_path)

def DirHasDirsInside(dir_path):
    """
    Checks if there are any directories inside a directory.

    Args:
        dir_path (str): Path of a directory to be checked.

    Returns:
        True/False
    """
    OGpath = getcwd()
    chdir(dir_path)
    nodelist = listdir()
    chdir(OGpath)
    for node in nodelist:
        if path.isdir(dir_path + "/" + node):
            return True
    return False

def GetAudiosFromCWD():
    """
    Returns a list of mp3 and flac files in current working directory.

    Returns:
        list (str): Names of mp3 and flac files in current working directory.
    """
    audios_in_cwd = []
    for node in listdir():
        if node.endswith(".mp3") or node.endswith(".flac"):
            audios_in_cwd.append(node)
    return audios_in_cwd

def GetDirsFromCWD():
    """
    Returns a list of directories in current working directory.

    Returns:
        list (str): Names of directories in current working directory.
    """
    dirs_in_cwd = []
    for node in listdir():
        if path.isdir(node):
            dirs_in_cwd.append(node)
    return dirs_in_cwd

def AddImageToAudiofilesInDir(album_path, image_path):
    """
    Adds an image to all mp3 and flac files inside a directory.

    Args:
        album_path (str): Path of a directory containing audio files.
        image_path (str): Path of an image to attribute.
    """
    OGpath = getcwd()
    chdir(album_path)
    songs_in_cd = GetAudiosFromCWD()
    chdir(OGpath)
    for audiofile in songs_in_cd:
        RemoveAndAddImageToAudiofile(album_path + "/" + audiofile, image_path)

def AttributeImagesRecursion(dir_path):
    """
    Recursively attributes images to songs.

    Function working order:
        If name of cwd is in images list, attribute image of this name to all audio files inside.
        If it's not, check if names of any audio files in cwd match names if images in image 
        list and attribute accordingly.
        Recur in every directory inside cwd.

    Args:
        dir_path (str): Path of a starting directory.
    """
    OGpath = getcwd()
    chdir(dir_path)
    CWDname = path.basename(getcwd())
    index = 0
    did_attribute = False
    
    
    while index < len(images_list):
        if CWDname == images_list_no_ext[index]:
            AddImageToAudiofilesInDir(getcwd(), images_path + "/" + images_list[index])
            images_list.pop(index)          ###### Picture can't be attributed to another album
            images_list_no_ext.pop(index)   ###### Picture can't be attributed to another album
            did_attribute = True
            break
        index += 1

    if not did_attribute:
        audios_in_cwd = GetAudiosFromCWD()
        for audioname in audios_in_cwd:
            if audioname[-4:] == "flac":
                audioname_no_ext = audioname[:-5]
            elif audioname[-3:] == "mp3":
                audioname_no_ext = audioname[:-4]
            index = 0
            while index < len(images_list):
                if audioname_no_ext == images_list_no_ext[index]:
                    RemoveAndAddImageToAudiofile(getcwd() + "/" + audioname, images_path + "/" + images_list[index])
                    images_list.pop(index)          ###### Picture can't be attributed to another album
                    images_list_no_ext.pop(index)   ###### Picture can't be attributed to another album
                    break
                index += 1
    
    
    dirs_in_cwd = GetDirsFromCWD()
    for direct in dirs_in_cwd:
        AttributeImagesRecursion(direct)

    chdir(OGpath)

def RemoveImageFromAudio(audio_path):
    if audio_path.endswith(".mp3"):
        RemoveImageMP3(audio_path)
    elif audio_path.endswith(".flac"):
        RemoveImageFLAC(audio_path)

def RemoveImagesRecursion(dir_path):
    OGpath = getcwd()
    chdir(dir_path)
    audios_list = GetAudiosFromCWD()

    for audio in audios_list:
        RemoveImageFromAudio(getcwd() + "/" + audio)

    dirs_in_cwd = GetDirsFromCWD()
    for direct in dirs_in_cwd:
        RemoveImagesRecursion(direct)

    chdir(OGpath)

# The recursive function doesn't change names of audiofiles in cwd and instead 
# has a function that changes is separately, because there would be a 
# significant time loss

In [13]:
try:
    audio_path
    images_path
    
except:
    pass

else:
    if path.isdir(audio_path):
        audio_path_isdir = True
    else:
        audio_path_isdir = False
    if path.isdir(images_path):
        chdir(images_path)
        images_list = [node for node in listdir() if (node[-4:] in [".png", ".jpg"] or node.endswith(".jpeg"))]
        images_list_no_ext = [image[:-4] if image[-4] == "." else image[:-5] for image in images_list]
        images_path_isdir = True
    else:
        images_path_isdir = False


    if audio_path_isdir == False and images_path_isdir == False:
        RemoveAndAddImageToAudiofile(audio_path, images_path)
        print(path.basename(audio_path))

    elif audio_path_isdir == True and images_path_isdir == False:
        AddImageToAudiofilesInDir(audio_path, images_path)

        # with the same name, any depth 
        # AttributeImagesRecursion(audio_path)              

    ##########Works until here
    else:
        if audio_path_isdir == False and images_path_isdir == True:
            index = 0
            audiofile_name = path.basename(audio_path)
            if audiofile_name.endswith(".jpeg"):
                audiofile_name_no_ext = audiofile_name[:-5]
            else:
                audiofile_name_no_ext = audiofile_name[:-4]

            while index < len(images_list_no_ext):
                if audiofile_name_no_ext == images_list_no_ext[index]:
                    print(audiofile_name)
                    RemoveAndAddImageToAudiofile(audio_path, images_path + "/" + images_list[index])
                    images_list.pop(index)          ###### Picture can't be attributed to another album
                    images_list_no_ext.pop(index)   ###### Picture can't be attributed to another album
                    break
                index += 1

        elif audio_path_isdir == True and images_path_isdir == True:
            AttributeImagesRecursion(audio_path)


try:
    del_path
except:
    pass
else:
    if del_path.endswith("mp3"):
        RemoveImageMP3(del_path)
    elif del_path.endswith("flac"):
        RemoveImageFLAC(del_path)
    elif path.isdir(del_path):
        RemoveImagesRecursion(del_path)